In [ ]:
import copy, sys, pandas as pd
import random
import time
import numpy as np
import subprocess 

sys.path.append("../../")
sys.path.append("../../inferring")
sys.path.append("../../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS

from inferring.InferringDFA import InferringDFA

In [2]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [16]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                 lstar_with_advice,
                 target=None,
                 ttt_eq=None, 
                 ttt_wa_eq=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.lstar_with_advice = lstar_with_advice
        self.ttt_eq = ttt_eq
        self.ttt_wa_eq = ttt_wa_eq 
        self.target = target

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.lstar_with_advice}"
        )
    def get_complete_description(self):
        return self.target.print_complete_description()
    
    def print_target(self):
        print(self.target.print_complete_description())

In [28]:
def getNumberofEQ(s):
    for l in s.splitlines():
        if l.startswith("Learning rounds"):
            return l.split()[-1]

In [ ]:
"""
    Files TTTExample1.java and TTTExample1_withAS.java should be placed in the following directory: examples/src/main/java/de/learnlib/example
    Before executing TTTExample1.java file build learlib project:                                   mvn clean install 
"""


number_of_itreations = 65

#  Fixed random seeds for reproducibility
seeds = [i for i in range(number_of_itreations)]

# The alphabet of the DFA
input_signs = ["a", "b", "c"]

# The bound on the number of states
max_number_of_states = 40

# The list to aggregate the results
results = []

_ = subprocess.run(["cd ../../../learnlib/examples ; mvn clean install; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
i = 0
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    # Create two random DFAs
    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )
    dfa2.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )

    # Run learning of the language of DFA to minimize dfa1 and dfa2
    d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    if d1.Q < 2 or d2.Q < 2:
        continue
        # Create two random DFAs
        dfa1 = DFA()
        dfa2 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )

        # Run learning of the language of DFA to minimize dfa1 and dfa2
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # Create a convolution DFA based on d1 and d2
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2)

    # Learn conv_dfa with advice, and store d - learned automaton 
    start_t = time.time()
    d, lstar_with_advice, lstar_ex_with_advice = run_learning_process(
        target=conv_dfa, advice_system=SRS(), check_consistency=True
    )
    end_t = time.time()

    print("ex time with AS: ", end_t - start_t)
    print(f"|d1.Q| = {d1.Q}, |d2.q| = {d2.Q}, |d.Q| = {d.Q}")
    d.type = DFA.CONV_DFA

    results.append(
        Record(
            d1=d1.Q,
            d2=d2.Q,
            conv=d.Q,
            lstar_with_advice=(lstar_with_advice, lstar_ex_with_advice),
            target=copy.deepcopy(d),
        )
    )
    results[-1].print_record()
    f = open("../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" + ".txt", "w")
    f.write(results[-1].get_complete_description())
    f.close()

    ttt_output = subprocess.run(["cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.TTTExample1\" ; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
    print("TTT done")
    ttt_wa_output = subprocess.run(["cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.TTTExample1_withAS\" ; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
    print("TTT with AS done")

    results[-1].ttt_eq = int(getNumberofEQ(ttt_output.stdout))
    results[-1].ttt_wa_eq = int(getNumberofEQ(ttt_wa_output.stdout))
    

iter nr: 1
ex time with AS:  19.26069211959839
|d1.Q| = 26, |d2.q| = 15, |d.Q| = 390
|d1| = 26, |d2| = 15, |conv| = 390, (mq, eq) = ([208929, 8], 158)
TTT done
-------------------------------------------------------
13:52:46.044 INFO     d.l.u.s.SimpleProfiler - Learning [ms]: 1104, (1.104 s) 
13:52:46.047 INFO     d.l.u.s.SimpleProfiler - Searching for counterexample [ms]: 151, (0.151 s) 
Learning rounds [#]: 382
Queries [#]: 100132
Symbols [#]: 930329
-------------------------------------------------------

TTT with AS done
Learning rounds: 21
nullQueries [#]: 93337
Symbols [#]: 1099259

iter nr: 2
iter nr: 3
iter nr: 4
ex time with AS:  3.5688116550445557
|d1.Q| = 18, |d2.q| = 16, |d.Q| = 288
|d1| = 18, |d2| = 16, |conv| = 288, (mq, eq) = ([67387, 6], 71)
TTT done
-------------------------------------------------------
13:52:58.194 INFO     d.l.u.s.SimpleProfiler - Learning [ms]: 720, (0.72 s) 
13:52:58.197 INFO     d.l.u.s.SimpleProfiler - Searching for counterexample [ms]: 126, (0

In [44]:
results.sort(key=lambda x: x.conv)
# Remove degenerated cases (Conv DFA of size 1)
results = [r for r in results if r.conv != 1]

print(f"number of tests: {len(results)}")
for i, r in enumerate(results):
    r.print_record()
    f = open("../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" + str(i) + ".txt", "w")
    f.write(results[i].get_complete_description())
    f.close()

number of tests: 39
|d1| = 17, |d2| = 15, |conv| = 255, (mq, eq) = ([62108, 3], 70)
|d1| = 13, |d2| = 20, |conv| = 260, (mq, eq) = ([70103, 3], 76)
|d1| = 14, |d2| = 19, |conv| = 266, (mq, eq) = ([91864, 7], 110)
|d1| = 17, |d2| = 16, |conv| = 272, (mq, eq) = ([77572, 5], 86)
|d1| = 21, |d2| = 13, |conv| = 273, (mq, eq) = ([60600, 2], 64)
|d1| = 18, |d2| = 16, |conv| = 288, (mq, eq) = ([67387, 6], 71)
|d1| = 12, |d2| = 25, |conv| = 300, (mq, eq) = ([106818, 10], 109)
|d1| = 21, |d2| = 15, |conv| = 315, (mq, eq) = ([78475, 6], 70)
|d1| = 16, |d2| = 20, |conv| = 320, (mq, eq) = ([102354, 4], 98)
|d1| = 15, |d2| = 24, |conv| = 360, (mq, eq) = ([100733, 2], 77)
|d1| = 19, |d2| = 19, |conv| = 361, (mq, eq) = ([109916, 4], 93)
|d1| = 19, |d2| = 20, |conv| = 380, (mq, eq) = ([150339, 4], 119)
|d1| = 16, |d2| = 24, |conv| = 384, (mq, eq) = ([236036, 4], 175)
|d1| = 24, |d2| = 16, |conv| = 384, (mq, eq) = ([157683, 3], 120)
|d1| = 24, |d2| = 16, |conv| = 384, (mq, eq) = ([136261, 3], 100)
|d1| 

In [47]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_eq_red = df[(('Reduction', 'EQ'))].mean()
    max_eq_red = df[(('Reduction', 'EQ'))].max()
    min_eq_red = df[(('Reduction', 'EQ'))].min()
    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v 

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    return df

In [48]:
ttt_eqs = [r.ttt_eq for r in results]
ttt_wa_eqs = [r.ttt_wa_eq for r in results]
red = ((np.array(ttt_eqs) - np.array(ttt_wa_eqs)) / np.array(ttt_eqs)) * 100

columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('TTT', 'EQ'), ('TTT with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: ttt_eqs,
    columns[4]: ttt_wa_eqs,
    columns[5]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

Target language            TTT TTT with advice Reduction
     conv(DFA1, DFA2) DFA1 DFA2   EQ              EQ        EQ
0                 255   17   15  249               8       96%
1                 260   13   20  248              15       93%
2                 266   14   19  259              14       94%
3                 272   17   16  268              14       94%
4                 273   21   13  268              16       94%
5                 288   18   16  277              15       94%
6                 300   12   25  293              14       95%
7                 315   21   15  305              13       95%
8                 320   16   20  313              20       93%
9                 360   15   24  336               5       98%
10                361   19   19  352              15       95%
11                380   19   20  369               9       97%
12                384   16   24  365              21       94%
13                384   24   16  375              14       96%
14                384   24   16  377              10       97%
15                390   26   15  382              21       94%
16                396   22   18  380               8       97%
17                420   28   15  405               6       98%
18                425   17   25  415              25       93%
19                425   17   25  405               7       98%
20                441   21   21  433              15       96%
21                459   27   17  456              23       94%
22                459   27   17  455              11       97%
23                476   17   28  466              12       97%
24                486   18   27  480              17       96%
25                504   24   21  460              13       97%
26                513   27   19  503               8       98%
27                525   25   21  519              16       96%
28                540   20   27  523               4       99%
29                540   20   27  527               3       99%
30                540   20   27  526               7       98%
31                550   22   25  524               9       98%
32                588   21   28  577               9       98%
33                600   24   25  570              16       97%
34                660   30   22  636              11       98%
35                702   26   27  689              14       97%
36                702   26   27  688              10       98%
37                720   24   30  704              22       96%
38                728   28   26  713               6       99%
39                                                            
40      Max reduction                                      99%
41      Min reduction                                      93%
42  Average reduction                                      96%